# Fine-tuning YOLOv8 on ImageNet (15 Categories)

## Objectif
Ce notebook effectue le fine-tuning du modèle YOLOv8n pré-entraîné sur un sous-ensemble de 15 catégories ImageNet pour améliorer la détection d'objets pour les 15 categories.

## Contenu
- Chargement et préparation des données ImageNet
- Configuration du modèle YOLOv8n
- Entraînement et validation
- Évaluation des performances

In [5]:
!pip install ultralytics opencv-python scikit-image pandas matplotlib


In [6]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, random


In [ ]:
ROOT = Path(r"C:\Users\admin\Documents\Master\S3\ComVision\mini projet 1")
DATA_ROOT = ROOT / "DATA"
YOLO_DATASET = DATA_ROOT / "dataset_yolo"

print("ROOT:", ROOT)
print("DATA_ROOT:", DATA_ROOT)
print("YOLO_DATASET:", YOLO_DATASET)

train_imgs = list((YOLO_DATASET / "images" / "train").glob("*.*"))
val_imgs   = list((YOLO_DATASET / "images" / "val").glob("*.*"))
train_lbls = list((YOLO_DATASET / "labels" / "train").glob("*.txt"))
val_lbls   = list((YOLO_DATASET / "labels" / "val").glob("*.txt"))

print("Nb images train :", len(train_imgs))
print("Nb labels train :", len(train_lbls))
print("Nb images val   :", len(val_imgs))
print("Nb labels val   :", len(val_lbls))


C:\Users\admin\Documents\Master\S3\ComVision\mini projet 1
C:\Users\admin\Documents\Master\S3\ComVision\mini projet 1/DATA
C:\Users\admin\Documents\Master\S3\ComVision\mini projet 1/DATA/dataset_yolo


In [ ]:
sample_label = random.choice(train_lbls)
print("Label choisi :", sample_label)

print(sample_label.read_text())

img_name = sample_label.stem
img_path = YOLO_DATASET / "images" / "train" / f"{img_name}.JPEG"
if not img_path.exists():
    # peut-être .jpg ou .png
    for ext in [".jpg", ".JPG", ".png"]:
        p = YOLO_DATASET / "images" / "train" / f"{img_name}{ext}"
        if p.exists():
            img_path = p
            break

print("Image associée :", img_path, img_path.exists())


In [ ]:
def show_yolo_bbox(img_path, label_path):
    img = cv2.imread(str(img_path))
    h, w = img.shape[:2]
    boxes = []
    for line in label_path.read_text().strip().splitlines():
        cid, xc, yc, bw, bh = map(float, line.split())
        xc *= w; yc *= h; bw *= w; bh *= h
        x1 = int(xc - bw/2); y1 = int(yc - bh/2)
        x2 = int(xc + bw/2); y2 = int(yc + bh/2)
        boxes.append((int(cid), x1, y1, x2, y2))

    img_vis = img.copy()
    for cid, x1, y1, x2, y2 in boxes:
        cv2.rectangle(img_vis, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(img_vis, str(int(cid)), (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    plt.figure(figsize=(4,4))
    plt.imshow(cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(img_path.name)
    plt.show()

show_yolo_bbox(img_path, sample_label)


In [ ]:
yaml_path = YOLO_DATASET / "dataset_yolo.yaml"

yaml_content = f"""
path: {YOLO_DATASET.as_posix()}
train: images/train
val: images/val

names:
  0: dog
  1: cat
  2: horse
  3: sheep
  4: bird
  5: chair
  6: bottle
  7: ball
  8: umbrella
  9: fork
  10: car
  11: motorcycle
  12:  airplane
  13: bicycle
  14: boat
"""

yaml_path.write_text(yaml_content)
print(yaml_path.read_text())
